In [1]:
import sys
sys.path.append('/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chemprop
import torch
from src.src import standardise_smiles
from matplotlib.offsetbox import AnchoredText

from sklearn.metrics import roc_auc_score, balanced_accuracy_score, confusion_matrix, cohen_kappa_score
from sklearn.decomposition import PCA

## Data Preprocessing

In [3]:
# read the author's data into dataframe
df=pd.read_csv(r"/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL/AID_1645848_datatable.csv")


In [4]:
# check the size of the data
df.shape

(2534, 11)

In [5]:
# check the first 5rows
print(df.head())

  PUBCHEM_RESULT_TAG  PUBCHEM_SID  PUBCHEM_CID  \
0        RESULT_TYPE          NaN          NaN   
1       RESULT_DESCR          NaN          NaN   
2                  1  387065710.0    5280445.0   
3                  2  174007039.0      72303.0   
4                  3  144206281.0    5388936.0   

                       PUBCHEM_EXT_DATASOURCE_SMILES PUBCHEM_ACTIVITY_OUTCOME  \
0                                                NaN                      NaN   
1                                                NaN                      NaN   
2      C1=CC(=C(C=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O)O                   Active   
3           C=CCC1=CC(=C(C=C1)O)C2=CC(=C(C=C2)O)CC=C                   Active   
4  CC[C@@H]1C(=O)N2CCC[C@H]2C(=O)N([C@H](C(=O)N3C...                   Active   

   PUBCHEM_ACTIVITY_SCORE  PUBCHEM_ACTIVITY_URL  PUBCHEM_ASSAYDATA_COMMENT  \
0                     NaN                   NaN                        NaN   
1                     NaN                   NaN         

In [6]:
# the analysis column classifies the compound as soluble or not
# class 0: moderate/ high solubility
# class 1: low solubility
df['Analysis Comment'].unique()

array(['STRING',
       "Annotation/notes on a particular compound's data or its analysis as described in the published paper.",
       'class = 0', 'class = 1', nan], dtype=object)

In [7]:
# check for duplicated data
df.duplicated().sum()

0

In [8]:
# check for missing values
df.isnull().sum()

PUBCHEM_RESULT_TAG                       0
PUBCHEM_SID                              2
PUBCHEM_CID                              2
PUBCHEM_EXT_DATASOURCE_SMILES            2
PUBCHEM_ACTIVITY_OUTCOME                 2
PUBCHEM_ACTIVITY_SCORE                   2
PUBCHEM_ACTIVITY_URL                  2534
PUBCHEM_ASSAYDATA_COMMENT             2534
Phenotype                               76
Kinetic Aqueous Solubility (ug/mL)      76
Analysis Comment                        76
dtype: int64

In [9]:
# Extract the relevant columns
relevant_columns = ['PUBCHEM_EXT_DATASOURCE_SMILES', 'Analysis Comment']
df = df[relevant_columns]

# Clean formatting issues (remove leading/trailing whitespaces)
df['PUBCHEM_EXT_DATASOURCE_SMILES'] = df['PUBCHEM_EXT_DATASOURCE_SMILES'].str.strip()

# Handle missing values (drop rows with missing values)
df = df.dropna()

df = df.drop_duplicates(keep='first')

# Convert molecules to standard SMILES
df['standardized_smiles'] = df['PUBCHEM_EXT_DATASOURCE_SMILES'].apply(standardise_smiles)

# Drop the original SMILES column
df.drop(columns=['PUBCHEM_EXT_DATASOURCE_SMILES'], inplace=True)

# Map the target variable to binary values
df['outcomes'] = df['Analysis Comment'].apply(lambda x: 1 if x == 'class = 1' else 0)

# Drop the original 'Analysis Comment' column
df.drop(columns=['Analysis Comment'], inplace=True)


# Rename the column
df = df.rename(columns={'PUBCHEM_EXT_DATASOURCE_SMILES': 'smiles'})

# path to save the cleaned author's data
output_folder = r'/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL'
output_filename = 'clean_author_data.csv'
output_path = f'{output_folder}/{output_filename}'

# Save the modified DataFrame back to a CSV file
df.to_csv(output_path, index=False)



In [10]:
df_gcnn= pd.read_csv(r"/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL/clean_author_data.csv")

In [11]:
# check the first 5 rows of cleaned data
print(df_gcnn.head())

                                 standardized_smiles  outcomes
0            O=c1cc(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12         0
1                C=CCc1ccc(O)c(-c2ccc(O)c(CC=C)c2)c1         0
2  CC[C@H]1NC(=O)[C@@H](NC(=O)c2ncccc2O)[C@@H](C)...         0
3     O=c1ncn2nc(Sc3ccc(F)cc3F)ccc2c1-c1c(Cl)cccc1Cl         0
4  O=C(Cc1ccc(Cl)c(Cl)c1)Nc1ccc(S(=O)(=O)Nc2ccon2...         0


In [12]:
df_gcnn.shape

(2455, 2)

# Train classification model
The author used the graph convolutional neural network available available through the ChemProp Python package (https://
github.com/chemprop/chemprop) by using default parameters.

In [13]:
# Define paths to your preprocessed dataset and where to save model checkpoints
data_path = r"/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL/clean_author_data.csv"
checkpoint_dir = r"/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL"



In [14]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Define command-line argument
arguments = [
    '--data_path', data_path,
    '--dataset_type', 'classification',
    '--save_dir', checkpoint_dir,
    '--epochs', '50',
    '--num_folds', '5',  # Perform 5-fold cross-validation
    '--save_smiles_splits',
    '--quiet'
]

# Parse arguments
args = chemprop.args.TrainArgs().parse_args(arguments)

# Run cross-validation
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

# Print mean and standard deviation of scores
print(f'Mean score: {mean_score:.4f}')
print(f'Standard deviation of score: {std_score:.4f}')


2455it [00:00, 134518.87it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 2455/2455 [00:01<00:00, 1582.20it/s]
Fold 0
0it [00:00, ?it/s]Warning: Repeated SMILES found in data, pickle file of split indices cannot distinguish entries and will not be generated.
1802it [00:00, 123956.70it/s]
 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:06<00:00,  6.26it/s]
                                                                                                                            
 68%|██████████████████████████████████████████████████████████▋                            | 27/40 [00:01<00:00, 20.92it/s]


 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:01<00:00, 21.22it/s]
                                                                                                                            
 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:01<00:00, 20.24it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 16.12it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:01<00:00, 19.59it/s]
                                                                                                                            


100%|███████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 19.92it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:01<00:00, 20.12it/s]
                                                                                                                            
 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:02<00:00, 19.78it/s]
                                                                                                                            
 15%|█████████████▏                                                                          | 6/40 [00:00<00:01, 19.50it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 21.47it/s]
                                                                                                                            
 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:02<00:00, 19.10it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:01<00:00, 19.24it/s]
                                                                                                                            
 65%|████████████████████████████████████████████████████████▌                              | 26/40 [00:01<00:00, 20.75it/s]


 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:01<00:00, 19.60it/s]
                                                                                                                            
 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:02<00:00, 19.33it/s]
                                                                                                                            
100%|███████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 21.77it/s]
                                                                                                                            
 65%|████████████████████████████████████████████████████████▌                              | 26/40 [00:01<00:00, 14.77it/s]


...

 98%|████████████████████████████████████████████████████████████████████████████████████▊  | 39/40 [00:03<00:00, 10.98it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 15.74it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 14.06it/s]
                                                                                                                            
 86%|██████████████████████████████████████████████████████████████████████████▊            | 43/50 [02:34<00:23,  3.42s/it]


 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 14.23it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 14.96it/s]
                                                                                                                            
 75%|█████████████████████████████████████████████████████████████████▎                     | 30/40 [00:02<00:00, 14.15it/s]


 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 14.62it/s]
                                                                                                                            
 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 14.56it/s]
                                                                                                                            
100%|███████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 11.15it/s]
                                                                                                                            
 25%|█████████████████████▊                                                                 | 10/40 [00:00<00:01, 15.03it/s]


 95%|██████████████████████████████████████████████████████████████████████████████████▋    | 38/40 [00:02<00:00, 15.05it/s]
                                                                                                                            
100%|███████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 14.49it/s]
                                                                                                                            
100%|███████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:55<00:00,  3.52s/it]
Model 0 best validation auc = 0.889249 on epoch 15
Model 0 test auc = 0.838650                                                                                                 
Ensemble test auc = 0.838650
5-fold cross validation
	Seed 0 ==> test auc = 0.858411
	Seed 1 ==> test auc = 0.888140
	Seed 2 ==> test auc = 0.872326
	Seed 3 ==> test auc = 0.889439
	Seed 4 ==> test a

Mean score: 0.8694
Standard deviation of score: 0.0191


In [ ]:
GCNN 0.90 0.80 0.71 0.90 0.59

In [50]:
result=pd.read_csv(r"/mnt/c/Users/HP/Desktop/Ersilia_ModelValidation/AUTHOR_GCNN_MODEL/test_scores.csv")
result

,Task,Mean auc,Standard deviation auc,Fold 0 auc,Fold 1 auc,Fold 2 auc,Fold 3 auc,Fold 4 auc
0,outcomes,0.869393,0.019119,0.858411,0.88814,0.872326,0.889439,0.83865
